根据下面的模型给出训练代码，数据集是{'sentence1':'xxx','sentence2':'yyy','label':1}{'sentence1':'xxx','sentence2':'yyy','label':0}的形式， sentence1对应文本，sentence2对应粗粒度主题分类或细粒度主题分类：
<复制sentence-transformer给的推理代码，让GPT修改成含有两个损失函数的训练方法>
提问：训练大模型时定义两个损失函数的训练方法

In [ ]:
# 下面是结合主题分类的主题标记训练方法的伪代码，由大模型生成。
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

# 加载预训练模型
model = SentenceTransformer('all-MiniLM-L6-v2')

# 定义数据集（示例数据，需替换为实际数据）
dataset = [
    {"sentence1": "机器学习提升效率", "sentence2": "人工智能", "label": 1},
    {"sentence1": "深度学习需要GPU", "sentence2": "硬件配置", "label": 0},
    # ...更多数据
]

# 数据预处理
train_examples = []
for item in dataset:
    # 将主题文本转换为分类标签（示例：假设主题到标签的映射）
    topic_to_label = {"人工智能": 0, "硬件配置": 1, "自然语言处理": 2}
    label = topic_to_label.get(item["sentence2"], -1)  # -1表示未知主题
    if label == -1:
        continue  # 忽略未知主题的数据
    train_examples.append(
        InputExample(
            texts=[item["sentence1"], item["sentence2"]],
            label=label
        )
    )

# 创建数据加载器
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# 定义损失函数（结合对比损失与分类损失）
class CombinedLoss(nn.Module):
    def __init__(self, alpha=0.5):
        super().__init__()
        self.cosine_loss = losses.CosineSimilarityLoss()
        self.classification_loss = nn.CrossEntropyLoss()
        self.alpha = alpha

    def forward(self, model_output, labels, attention_mask):
        # 对比学习部分（句子对相似度）
        anchor_emb = model_output[0]
        positive_emb = model_output[1]
        negative_emb = model_output[2] if len(model_output) > 2 else None
        similarity = self.cosine_loss(anchor_emb, positive_emb, negative_emb)
        
        # 分类任务部分（主题分类）
        pooled_emb = mean_pooling(model_output, attention_mask)
        class_logits = model.classifier(pooled_emb)  # 假设模型有classifier层
        class_loss = self.classification_loss(class_logits, labels)
        
        # 组合损失
        total_loss = self.alpha * similarity + (1 - self.alpha) * class_loss
        return total_loss

# 初始化优化器
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# 训练循环
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        
        # 前向传播
        inputs = tokenizer(batch.texts, padding=True, truncation=True, return_tensors="pt")
        outputs = model(**inputs, return_dict=True)
        loss_fn = CombinedLoss(alpha=0.7)  # 调整对比与分类的权重
        loss = loss_fn(outputs, batch.label, inputs["attention_mask"])
        
        # 反向传播
        loss.backward()
        optimizer.step()
        
        if step % 100 == 0:
            print(f"Epoch {epoch}, Step {step}, Loss: {loss.item():.4f}")

# 保存微调后的模型
model.save_pretrained("fine-tuned-sentence-transformer")